In [ ]:
from m2_cw import load_qwen, convert_tokens_to
from m2_cw import load_and_preprocess
import torch


train_texts, val_texts = load_and_preprocess("data/lotka_volterra_data.h5")

full_model, _ = load_qwen(small_vocabulary=False)
reduced_model, tokenizer, token_map = load_qwen(small_vocabulary=True)

device = torch.device("cpu")
full_model.to(device)
reduced_model.to(device)

print(full_model.device)
print(reduced_model.device)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


cpu
cpu


In [2]:
print(full_model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [3]:
print(reduced_model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(13, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbeddin

In [8]:
# Test of the reduced model's consistency

text = train_texts[0]

# Tokenize
full_tokens = tokenizer(text, return_tensors="pt", add_special_tokens=False)
reduced_tokens = convert_tokens_to("forecast", full_tokens, token_map)
num_input_tokens = full_tokens.input_ids.shape[1]

# Generate text with full model
full_output = full_model.generate(**full_tokens, max_new_tokens=20, do_sample=False)
full_new_tokens = full_output[:, num_input_tokens:]
full_generated_text = tokenizer.batch_decode(full_new_tokens, skip_special_tokens=True)

# Generate text with reduced model
reduced_output = reduced_model.generate(**reduced_tokens, max_new_tokens=20, do_sample=False)
reduced_new_tokens = reduced_output[:, num_input_tokens:]
reduced_new_tokens = convert_tokens_to("qwen", reduced_new_tokens, token_map)
reduced_generated_text = tokenizer.batch_decode(reduced_new_tokens, skip_special_tokens=True)

print(full_generated_text)
print(reduced_generated_text)

Got a batch encoding


/Users/chrismiller/venvs/m2_venvs/m2_coursework/lib/python3.13/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/Users/chrismiller/venvs/m2_venvs/m2_coursework/lib/python3.13/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/Users/chrismiller/venvs/m2_venvs/m2_coursework/lib/python3.13/site-packages/transformers/generation/configuration_utils.py:651: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warni

Got a tensor
['7.01,1.28;7.83,1.24;']
['7.01,1.28;7.83,1.24;']
